<a href="https://colab.research.google.com/github/atomicamit24/Minor-Projects/blob/main/Simple_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Fix the provided Python code for a chatbot, integrate the WeatherAPI using the provided API key "428db8443c15458cb9955436252507" and API URL "https://www.weatherapi.com/my/", and add the following features: remember user names, integrate advanced NLP, develop contextual understanding, add new API integrations (dictionary and translation), enhance sentiment analysis, explore basic learning mechanisms, introduce different response styles, implement user profiles, improve error handling and fallbacks, and add persistent storage.

## Refactor and modularize

### Subtask:
Restructure the existing code to make it more modular and easier to add new features. This might involve creating separate functions or classes for different functionalities like NLP, API calls, and response generation.


**Reasoning**:
Restructure the existing code into a `Chatbot` class with methods for different functionalities, including initializing parameters, preprocessing input, extracting city information, getting weather data, and generating responses. This addresses steps 1, 2, 3, 4, and 5 of the instructions.



In [ ]:
import re
import requests
import datetime

# Define the Chatbot class to encapsulate all chatbot functionalities
class Chatbot:
    def __init__(self, api_key):
        # Initialize the chatbot with the provided API key for external services
        self.api_key = api_key
        # Set the base URL for the WeatherAPI
        self.weather_api_url = "http://api.weatherapi.com/v1/current.json"
        # Define a dictionary of standard responses
        self.responses = {
            "hello": "Hi there! How can I help you today?",
            "hi": "Hello! What can I do for you?",
            "hey": "Hey! How's it going?",
            "how are you": "I'm just a program, but I'm doing great! Thanks for asking.",
            "what is your name": "I am a simple chatbot, designed to assist you.",
            "bye": "Goodbye! Have a great day!",
            "thanks": "You're welcome!",
            "thank you": "You're very welcome!",
            "help": "I can answer basic questions about myself, greetings, and provide weather for a city.",
            "who created you": "I was created by a developer for demonstration purposes.",
            # Use lambda for dynamic responses like time and date
            "time": lambda: f"The current time is {datetime.datetime.now().strftime('%I:%M %p')}.",
            "date": lambda: f"Today's date is {datetime.datetime.now().strftime('%Y-%m-%d')}.",
            "how old are you": "I don't have an age, as I'm just a computer program.",
            "what do you do": "I can answer simple questions, provide greetings, and give weather updates for a specified city.",
            "what is your purpose": "My purpose is to demonstrate basic chatbot functionalities, including NLP and API integration."
        }
        # Define patterns for basic intent recognition
        self.patterns = {
            "hi": "greeting",
            "hello": "greeting",
            "hey": "greeting",
            "how are you": "greeting_how_are_you",
            "your name": "basic_info_name",
            "who created you": "basic_info_creator",
            "time": "basic_info_time",
            "date": "basic_info_date",
            "old are you": "basic_info_age",
            "do you do": "basic_info_purpose",
            "your purpose": "basic_info_purpose",
            "help": "basic_info_help",
            "what can you do": "basic_info_help",
            "weather": "weather_query",
            "temperature": "weather_query",
            "forecast": "weather_query",
            "climate": "weather_query",
            "how hot": "weather_query",
            "how cold": "weather_query",
            "happy": "sentiment_positive",
            "good": "sentiment_positive",
            "great": "sentiment_positive",
            "sad": "sentiment_negative",
            "bad": "sentiment_negative",
            "unhappy": "sentiment_negative",
            "bye": "farewell",
            "goodbye": "farewell",
            "see you": "farewell",
            "thank": "acknowledgement"
        }
        # A list of common cities to help with city extraction
        self.common_cities = ["london", "paris", "new york", "tokyo", "delhi", "mumbai", "kolkata", "chennai", "bengaluru", "pune", "hyderabad", "chicago", "sydney", "berlin", "moscow", "rome", "dubai", "singapore", "hong kong", "san francisco", "los angeles"]

    # Method to preprocess user input: lowercasing, removing punctuation, and tokenization
    def preprocess_input(self, user_input):
        user_input = user_input.lower()
        user_input = re.sub(r'[^\w\s]', '', user_input)
        user_input = re.sub(r'\s+', ' ', user_input).strip()
        tokens = user_input.split()
        return tokens

    # Method to extract city names from the user input
    def extract_city(self, user_input_tokens):
        text_str = " ".join(user_input_tokens)

        # First, check for common cities
        for city in self.common_cities:
            if city in text_str:
                return city.title()

        # Then, use regex to find potential cities based on common patterns
        # Pattern: "in [city name]"
        match_in = re.search(r'in\s+([a-zA-Z\s]+?)(?:\s+weather|\s*[\.\?!,]|$)', text_str)
        if match_in:
            potential_city = match_in.group(1).strip()
            # Basic validation for the extracted potential city
            if potential_city and potential_city not in ["the", "this", "that", "my", "your"]:
                if len(potential_city.split()) <= 3 and all(word.isalpha() for word in potential_city.split()):
                    return potential_city.title()

        # Pattern: "[city name] weather"
        match_weather_before = re.search(r'([a-zA-Z\s]+?)\s+weather', text_str)
        if match_weather_before:
            potential_city = match_weather_before.group(1).strip()
            # Basic validation for the extracted potential city
            if potential_city and potential_city not in ["what", "how", "is", "the"]:
                 if len(potential_city.split()) <= 3 and all(word.isalpha() for word in potential_city.split()):
                    return potential_city.title()

        # Fallback: If no specific pattern matches, consider the entire input (if not too short) as a potential city
        if len(user_input_tokens) > 0:
            potential_city = " ".join(user_input_tokens)
            if potential_city and re.search(r'[a-zA-Z]', potential_city):
                 # Remove trailing punctuation
                 potential_city = re.sub(r'[\.\?!,]\s*$', '', potential_city).strip()
                 # Avoid common non-city words
                 if potential_city.lower() not in ["the", "a", "is", "in", "of", "for", "what", "how"]:
                    return potential_city.title()

        # Return None if no city is found
        return None

    # Method to fetch current weather data from the WeatherAPI
    def get_current_weather(self, city_name):
        # Check if API key is configured
        if not self.api_key or self.api_key == 'YOUR_OPENWEATHERMAP_API_KEY':
            return "Weather service not configured. Please get a WeatherAPI.com API key and update the code."

        # Prepare parameters for the API request
        params = {
            'q': city_name,
            'key': self.api_key,
        }
        try:
            # Make the API call
            response = requests.get(self.weather_api_url, params=params)
            # Raise an HTTPError for bad responses (4xx or 5xx)
            response.raise_for_status()
            # Parse the JSON response
            data = response.json()

            # Extract and format weather information
            if 'current' in data and 'location' in data:
                location = data['location']
                current = data['current']
                city = location['name']
                country = location['country']
                temp_c = current['temp_c']
                feelslike_c = current['feelslike_c']
                humidity = current['humidity']
                condition = current['condition']['text']
                return (f"The weather in {city}, {country} is currently {condition}."
                        f" The temperature is {temp_c:.1f}°C, but it feels like {feelslike_c:.1f}°C."
                        f" Humidity is {humidity}%.")
            # Handle API-specific errors
            elif 'error' in data:
                 return f"Sorry, I couldn't find weather information for {city_name}. Error: {data['error']['message']}"
            # Handle unexpected API responses
            else:
                 return f"Received an unexpected response from the weather service for {city_name}."

        # Implement comprehensive error handling for network and API issues
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 401:
                return "My weather service is currently unauthorized. Please check your WeatherAPI.com API key in the code."
            else:
                return f"An HTTP error occurred while fetching weather data: {e}"
        except requests.exceptions.ConnectionError:
            return "I'm having trouble connecting to the weather service. Please check your internet connection."
        except requests.exceptions.Timeout:
            return "The weather service took too long to respond. Please try again later."
        except requests.exceptions.RequestException as e:
            return f"An error occurred with the weather request: {e}"
        except Exception as e:
            return f"An unexpected error occurred: {e}"

    # Method to generate a bot response based on user input
    def get_bot_response(self, user_input):
        # Preprocess the input
        processed_tokens = self.preprocess_input(user_input)
        processed_input_string = " ".join(processed_tokens)

        # Check for explicit 'bye' command
        if processed_input_string == 'bye':
            return "farewell_signal"

        # Check for direct matches in the responses dictionary
        if processed_input_string in self.responses:
            response = self.responses[processed_input_string]
            # Execute lambda functions if the response is callable
            if callable(response):
                return response()
            return response

        # Infer intent based on predefined patterns
        inferred_intent = None
        for pattern_key, intent_type in self.patterns.items():
            if pattern_key in processed_input_string:
                inferred_intent = intent_type
                break

        # Generate response based on the inferred intent
        if inferred_intent == "greeting":
            return "Hello there! How can I assist you?"
        elif inferred_intent == "greeting_how_are_you":
            return self.responses["how are you"]
        elif inferred_intent == "basic_info_name":
            return self.responses["what is your name"]
        elif inferred_intent == "basic_info_creator":
            return self.responses["who created you"]
        elif inferred_intent == "basic_info_time":
            return self.responses["time"]()
        elif inferred_intent == "basic_info_date":
            return self.responses["date"]()
        elif inferred_intent == "basic_info_age":
            return self.responses["how old are you"]
        elif inferred_intent == "basic_info_purpose":
            return self.responses["what do you do"]
        elif inferred_intent == "basic_info_help":
            return self.responses["help"]
        elif inferred_intent == "acknowledgement":
            return self.responses["thanks"]
        elif inferred_intent == "farewell":
            return self.responses["bye"]
        elif inferred_intent == "sentiment_positive":
            return "That's wonderful to hear!"
        elif inferred_intent == "sentiment_negative":
            return "I'm sorry to hear that. I hope things get better for you."
        elif inferred_intent == "weather_query":
            # Extract city for weather queries and get weather data
            city = self.extract_city(processed_tokens)
            if city:
                return self.get_current_weather(city)
            else:
                return "Which city's weather would you like to know? Please specify, e.g., 'weather in London'."

        # Default response for unrecognized input
        return "I'm sorry, I don't understand that. Can you please rephrase or ask about weather for a city?"

    # The main chat loop
    def chat(self):
        print("Chatbot: Hi! I'm an enhanced chatbot. I can greet, answer basic questions, and give weather updates.")
        print("Chatbot: Try asking about 'time', 'your name', or 'weather in London'.")
        print("Chatbot: Type 'bye' to exit.")
        while True:
            try:
                user_input = input("You: ")
                # Exit condition for the chat loop
                if user_input.lower() == 'bye':
                    print("Chatbot: Goodbye! Have a great day!")
                    break

                # Get the bot's response
                response = self.get_bot_response(user_input)

                # Handle farewell signal for exiting the loop
                if response == "farewell_signal":
                    print("Chatbot: Goodbye! Have a great day!")
                    break

                # Print the bot's response
                print(f"Chatbot: {response}")
            # Handle KeyboardInterrupt (e.g., Ctrl+C)
            except KeyboardInterrupt:
                print("\nChatbot: Chat interrupted by user. Goodbye!")
                break
            # Catch any other unexpected errors during the chat
            except Exception as e:
                print(f"Chatbot: An unexpected error occurred: {e}")
                print("Chatbot: Restarting chat loop.")

# --- Start the Chatbot ---
# This block ensures the chat starts only when the script is executed directly
if __name__ == "__main__":
    # Create an instance of the Chatbot with the specified API key
    chatbot_instance = Chatbot(api_key="428db8443c15458cb9955436252507")
    # Start the chat
    chatbot_instance.chat()

Chatbot: Hi! I'm an enhanced chatbot. I can greet, answer basic questions, and give weather updates.
Chatbot: Try asking about 'time', 'your name', or 'weather in London'.
Chatbot: Type 'bye' to exit.
You: weather in asansol
Chatbot: The weather in Asansol, India is currently Partly Cloudy. The temperature is 26.4°C, but it feels like 29.9°C. Humidity is 86%.
You: hi my name is amit
Chatbot: Hello there! How can I assist you?
You: what you can do
Chatbot: I'm sorry, I don't understand that. Can you please rephrase or ask about weather for a city?
You: help me
Chatbot: I can answer basic questions about myself, greetings, and provide weather for a city.
You: greetings
Chatbot: I'm sorry, I don't understand that. Can you please rephrase or ask about weather for a city?
You: ok
Chatbot: I'm sorry, I don't understand that. Can you please rephrase or ask about weather for a city?
You: bye
Chatbot: Goodbye! Have a great day!


# Task
Integrate advanced NLP techniques into the provided Python chatbot code to improve intent recognition and entity extraction, using the API key "428db8443c15458cb9955436252507" and the API URL "https://www.weatherapi.com/my/".

## Choose and install nlp library

### Subtask:
Select an NLP library (e.g., spaCy or NLTK) and add the necessary code to install it in the notebook.


**Reasoning**:
Install the spaCy library using pip to prepare for integrating advanced NLP techniques.



In [ ]:
%pip install spacy

## Integrate advanced NLP

### Subtask: Choose and Install NLP Library

Select an NLP library (e.g., spaCy or NLTK) and add the necessary code to install it in the notebook.

**Reasoning**:
Install the spaCy library using pip to prepare for integrating advanced NLP techniques.

In [ ]:
%pip install spacy

### Subtask: Load NLP Model

Add code to load the required language model from the chosen library.

**Reasoning**:
Load the 'en_core_web_sm' language model from spaCy to enable linguistic processing.

In [ ]:
import spacy

# Download the smaller English language model
!python -m spacy download en_core_web_sm

# Load the model
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 105.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
